### 一、Preprocess and generate dataset

In [14]:
import pandas as pd
import os,glob
import numpy as np
from copy import deepcopy
import SimpleITK as sitk
from scipy.ndimage import binary_fill_holes
import torch
from matplotlib import pyplot as plt 
from PIL import Image
import imageio
from batchgenerators.utilities.file_and_folder_operations import maybe_mkdir_p, join

In [ ]:
def maybe_create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return path

In [13]:
#!pip install imageio

In [15]:
csv = '/mnt/f/data/CellPress7/instances_copy.csv'
original_data_dir = '/mnt/f/data/CellPress7'
brain_dst_dataset = '/mnt/f/data/CellPress7/brain'

In [17]:
def save_itk_from_numpy(numpy_data, origin_itk_img):
    pred_itk_image = sitk.GetImageFromArray(numpy_data)
    pred_itk_image.CopyInformation(origin_itk_img)
    return pred_itk_image

In [27]:
def generate_brain_mask(brain_matlab_path,save_f):
    
    # 值为1并且没有hole
    brain_nii = sitk.ReadImage(brain_matlab_path)
    brain_np = sitk.GetArrayFromImage(brain_nii).astype(np.int32)
    brain_mask = brain_np != 0
    nonhole_mask_bool = binary_fill_holes(brain_mask)
    nonhole_mask_01 = nonhole_mask_bool.astype(int)
    
    # 保存最大强度投影
    for i in range(3):
        mask_mip = np.max(nonhole_mask_01, axis=i)
        imageio.imsave(save_f + '_mip' + str(i) + '.png', mask_mip)

In [28]:
def generate_bone_window(cta_image,save_path):

    cta_nii = sitk.ReadImage(cta_image)
    cta_np = sitk.GetArrayFromImage(cta_nii).astype(np.float32)

    cta_tensor = torch.from_numpy(cta_np)
    bone_hu_channel = torch.clamp(cta_tensor, 100, 800)
    bone_image = (bone_hu_channel - 100) / (100 - 800)
    
    bone_image = bone_image.numpy()
    bone_image = abs(bone_image)
    bone_image = (bone_image - np.min(bone_image)) / (np.max(bone_image) - np.min(bone_image))
    
    for i in range(3):
        bone_mip = np.max(bone_image, axis=i)
        imageio.imsave(save_path + '_mip' + str(i) + '.png', bone_mip)

In [29]:
patient_names=[]
datalist = pd.read_csv(csv)
for i in range(len(datalist)):
    patient_name = datalist.iloc[i]['instance_id']
    patient_names.append(patient_name)

In [30]:
brain_ids = []

for name in patient_names:
    brain_matlab_path = os.path.join(brain_dst_dataset,'brain_matlab',name+'__brain.nii.gz')
    bone_cta_save = os.path.join(brain_dst_dataset,'precessed_bone_cta',name)
    if not os.path.exists(bone_cta_save):
        cta_image = os.path.join(original_data_dir,'cta_img',name+'.nii.gz')
        generate_bone_window(cta_image,bone_cta_save)
    brain_mask_save = os.path.join(brain_dst_dataset,'processed_brain_mask',name)
    if os.path.exists(brain_matlab_path) and (not os.path.exists(brain_mask_save)):
        generate_brain_mask(brain_matlab_path,brain_mask_save)
        brain_ids.append(name)

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0